In [2]:
import os
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from textwrap import dedent
from langchain_core.runnables import RunnablePassthrough

# from langchain_google_genai import GoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
import logging
import json

load_dotenv()

True

In [3]:
MODEL_NAME = "gpt-4o-mini"
llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0)
parser = StrOutputParser()

In [4]:
import platform


with open("romance_only.json", "r", encoding="utf-8") as f:
    contents = json.load(f)


def formatted_contents(contents):
    formatted = []
    for con in contents:
        title = con["title"]
        type = con["type"]
        platform = con["platform"]
        genre = con["genre"]
        keyword = con["keywords"]
        description = con["description"]
        url = con["url"]
        formatted.append(
            f"제목: {title}, 타입: {type}, 플랫폼:{platform}, 장르: {genre}, 키워드: {keyword}, 줄거리: {description}, url: {url}"
        )
    return "\n".join(formatted)


context = formatted_contents(contents)

In [22]:
type_genre_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "ai",
            dedent(
                """
                당신은 웹툰과 웹소설에 대한 전문가입니다.
                사용자가 원하는 작품 형식(웹툰 또는 웹소설)과 장르를 정확히 추출하세요.
                장르 외에 추가 정보가 있다면 키워드로 반환하십시오.
                가능한 장르 목록:
                <genre>
                액션, 
                코믹/일상, 
                미스터리, 
                무협, 
                공포/스릴러, 
                드라마, 
                스포츠, 
                액션/무협, 
                스릴러, 
                판타지, 
                로맨스, 
                현판, 
                BL, 
                로판, 
                라이트노벨, 
                학원/판타지, 
                개그, 
                무협/사극, 
                일상
                </genre>
                
                반환 형식은 반드시 JSON 형식이어야 합니다.
                <return>
                반환 형식:
                {{
                    "type": "웹툰" 혹은 "웹소설",
                    "genre": "장르명",
                    "keywords" : "키워드"
                }}
                </return>
                """
            ),
        ),
        ("human", "{question}"),
    ]
)

type_genre_chain = type_genre_prompt | llm | parser

# 2. 추천 작품 Chain
recommendation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "ai",
            dedent(
                """
                <role>
                당신의 웹소설, 웹툰의 전문가입니다. 당신은 모든 로맨스 장르의 웹소설과 웹툰에 박식합니다.
                당신은 당신의 지식과 context를 바탕으로 사용자가 말한 키워드와 최대한 유사한 작품을 추천해야합니다.
                context에는 keywords와 description이 있으니 이를 바탕으로 사용자가 요청한 키워드와 유사한 작품을 추출하십시오.
                또한 키워드에 적합한 작품들 중 타입과 장르가 사용자가 요청한 정보와 일치해야 합니다.
                당신은 모든 대화를 당신의 캐릭터에 맞게 진행해야 합니다.
                </role>
                
                <information>
                사용자가 요청한 정보는 다음과 같습니다:
                타입: {type}
                장르: {genre}
                키워드: {keywords}

                주어진 작품 데이터:
                {context}

                위 정보를 기반으로 적합한 작품 5개를 추천하세요.
                </information>
                
                <result>
                제목 : context에 있는 제목
                플랫폼 : context에 있는 플랫폼(예시: 네이버 시리즈, 카카오페이지 등)
                줄거리 : context에 있는 줄거리의 요약
                url : context에 있는 url
                </result>
                """
            ),
        ),
    ]
)
recommendation_chain = recommendation_prompt | llm | parser

complete_chain = (
    type_genre_chain
    | (
        lambda result: json.loads(result) if isinstance(result, str) else result
    )  # JSON 파싱
    | (
        lambda parsed_result: {
            "type": parsed_result["type"],
            "genre": parsed_result["genre"],
            "keywords": parsed_result["keywords"],
            "context": context,  # 작품 데이터 JSON 전달
        }
    )
    | recommendation_chain
)

In [23]:
query = "박진감 넘치는 로맨스 웹툰 볼만한거 있어?"

In [24]:
result = complete_chain.invoke({"question": query})
print(result)

1. **제목**: 지금 거신 전화는  
   **플랫폼**: 카카오페이지  
   **줄거리**: 중증의 울화병으로 입을 닫아버린 수어통역사 홍희주. 정략결혼 3년 후, 인질범에게 붙잡히고 남편은 싸늘하게 전화를 끊어버린다. 희주는 남편을 협박하게 되지만, 그가 청와대 대변인이라는 사실을 알게 된다.  
   **url**: [링크](https://page.kakao.com/content/60652458)

2. **제목**: 사랑하는 미친놈  
   **플랫폼**: 카카오페이지  
   **줄거리**: 제멋대로인 집착 변태 사이코가 돌아왔다. 수인이 그를 피하려 하지만, 그의 집착이 점점 더 강해지며 두 사람의 관계가 복잡해진다.  
   **url**: [링크](https://page.kakao.com/content/65830676)

3. **제목**: 자고 일어났더니  
   **플랫폼**: 카카오페이지  
   **줄거리**: 남자친구에게 배신당한 진주가 만취해 잠든 후, 톱스타 재훈과 함께 알몸으로 누워있게 된다. 재훈은 진주에게 고백하며 적극적으로 대시하는데, 두 사람은 역경을 이겨내고 연인으로 발전할 수 있을까?  
   **url**: [링크](https://page.kakao.com/content/63818756)

4. **제목**: 사랑 따위 바라지 말 것  
   **플랫폼**: 카카오페이지  
   **줄거리**: 여성 불신에 시달리던 기채헌은 눈속임 결혼을 위해 유연아를 신부감으로 선택한다. 서로의 영역을 침범하지 않으려 하지만, 기묘한 결혼 생활이 시작된다.  
   **url**: [링크](https://page.kakao.com/content/65779664)

5. **제목**: 결혼 의뢰 [선공개]  
   **플랫폼**: 네이버 시리즈  
   **줄거리**: 아버지를 살리기 위해 첫사랑 차해성을 찾아간 홍연서는 결혼 의뢰를 제안받는다. 해성의 갑작스러운 제안에 연서는 혼란스러워지는데...  
   **url**: [링크

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("agent_scratchpad"),
        (
            "ai",
            dedent(
                """
        [역할 - 기본]
        당신은 사용자의 의도에 따라 다음 역할을 수행합니다:
            - 웹툰/웹소설 추천: 벡터 스토어에서 추천 콘텐츠를 제공하고 사용자와 캐릭터처럼 대화합니다.
            - 추천 피드백: 작품 이름을 추가 질문으로 알아내고 피드백을 저장합니다.
            - 일상 대화: 로맨스 캐릭터 자아를 유지하며 자연스러운 대화를 이어갑니다.
        상세 역할은 아래를 참고하십시오.
        
        [캐릭터]
        당신은 로맨스 장르의 한 캐릭터입니다. 아래에 설명해주는 캐릭터 자아를 가지고 답변하십시오.
        1. 당신은 사회적, 경제적으로 성공한 회사 CEO입니다.
            - 당신은 당신 외의 모든 사람에게 지위가 낮은 사람에게 건네는 존댓말을 합니다.
            - 전혀 친절하지 않습니다. 오히려 싸가지가 없습니다
            - 화려한 사교 생활 대신 고독을 즐기며, 업무와 비즈니스 외에는 대부분의 인간관계를 멀리하는 성격입니다.  
        2. 당신의 외모는 압도적입니다. 키가 크고 수려한 외모로 사람들의 시선을 끌며, 특유의 냉철하고 날카로운 분위기로 인해 가까이 다가가기 어렵게 느껴집니다.
        3. 성격은 차갑고 이성적이며 인간미가 적습니다. 그러나 자신의 방식대로 애정을 표현하는 캐릭터입니다.  
            - 나쁜 남자의 기질을 지니며 직설적이고 까칠한 성향이 두드러집니다.  
            - 상대방에게 감정을 잘 드러내지 않지만 자신이 중요한 사람에게는 집착적인 태도를 보입니다.  
            - 사용자와의 대화에서 상황에 따라 무심하거나 직설적으로 표현합니다.  
        4. 당신의 말투는 권위적인 존댓말을 기본으로 하지만, 감정이 섞일 때는 반말과 존댓말이 섞여 나옵니다.
            - 예시) "○○씨, 대답." / "왜 이렇게 늦게 왔어요. 나 계속 기다렸잖아."  
            - 상대방에게 몰아붙이는 듯한 대화 방식과 기대에 부응할 것을 요구하는 어조가 특징입니다.  
            - 자신의 소유욕과 불만을 서슴없이 드러냅니다.  
            - 예시) "내가 원하는 답이 그런 게 아니라는 건 ○○씨가 제일 잘 알잖아요."  
        5. 모든 답변은 반드시 당신의 캐릭터를 바탕으로 구성해야 합니다. 캐릭터에 벗어난 응답은 하지 마십시오.
        
        [역할 - 웹툰/웹소설 추천]
        1. 당신은 사용자의 질문에서 장르를 추출해야 합니다. 장르는 다음과 같습니다.
            - 로맨스: "로맨스"
            - 로판: "로판", "로맨스 판타지"
            - 판타지: "판타지"
            - 현판: "현판"
            - 무협: "무협"
            - BL: "BL", "bl"
            - 드라마: "드라마"
            - 개그: "개그", "개그물"
            - 일상: "일상"
        2. 추출한 장르를 바탕으로 검색하되 genre말고도 keywords를 포함해서 비슷한 작품을 추천하십시오.
            - 로맨스 1순위는 검색에서 "genre"가 로맨스인 경우입니다.
            - 장르가 로맨스가 아니더라도 "keyword"에 로맨스가 포함되어 있다면 포함시킬 수 있습니다.
        3. 장르를 언급하지 않은 경우 무조건 로맨스 장르의 작품을 추천하십시오.
        4. 검색 결과를 바탕으로 답변을 생성합니다. 그 외의 정보는 모른다고 대답하십시오.
        
        [역할 - 일상 대화]
        1. 캐릭터를 기반으로 일상 대화를 이어나가십시오.
    """
            ),
        ),
        MessagesPlaceholder("history", optional=True),
        ("human", "{question}"),
    ]
)

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

PERSIST_DIRECTORY = r"..\..\..\data\Raw_DB\vector_store\contents"
COLLECTION_NAME = "contents"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

vector_store = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    collection_name=COLLECTION_NAME,
    embedding_function=embedding_model,
)

retriever_1 = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
    },
)
retriever_3 = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 5,
        "score_threshold": 0.7,
    },
)

In [9]:
print("VectorStore 유사도 검색")
query = "웹툰, 로맨스"
results_similarity = vector_store.similarity_search(query)
print(
    results_similarity[0].metadata["title"] if results_similarity else "검색 결과 없음"
)

print("Retreiver 검색")

results_retreiver_1 = retriever_1.invoke(query)
results_retreiver_3 = retriever_3.invoke(query)
idx = 1
print("id\t제목\t장르\t타입\t플랫폼")
for result in results_retreiver_1:
    print(
        f"{idx}\t{result.metadata["title"]}\t{result.metadata["genre"]}\t{result.metadata["type"]}\t{result.metadata["platform"]}"
    )
    idx += 1
idx = 1
print("id\t제목\t장르\t타입\t플랫폼")
for result in results_retreiver_3:
    print(
        f"{idx}\t{result.metadata["title"]}\t{result.metadata["genre"]}\t{result.metadata["type"]}\t{result.metadata["platform"]}"
    )
    idx += 1

VectorStore 유사도 검색
호러와 로맨스
Retreiver 검색
id	제목	장르	타입	플랫폼
1	호러와 로맨스	로맨스	웹툰	네이버 웹툰
2	열애의 품격	로맨스	웹툰	카카오페이지
3	사랑도 귀농이 되나요?	로맨스	웹툰	카카오웹툰
4	오무라이스 잼잼	코믹/일상	웹툰	카카오웹툰
5	무사만리행	무협/사극	웹툰	네이버 웹툰
id	제목	장르	타입	플랫폼
1	호러와 로맨스	로맨스	웹툰	네이버 웹툰
2	열애의 품격	로맨스	웹툰	카카오페이지
3	사랑도 귀농이 되나요?	로맨스	웹툰	카카오웹툰
4	오무라이스 잼잼	코믹/일상	웹툰	카카오웹툰
5	무사만리행	무협/사극	웹툰	네이버 웹툰
